In [20]:
from qcodes.instrument_drivers.tektronix.Keithley_s46 import S46, LockAcquisitionError

In [22]:
s46 = S46("s2", "GPIB0::7::INSTR")

Connected to: KEITHLEY INSTRUMENTS INC. SYSTEM 46 (serial:1327388, firmware:A03) in 0.20s


In [3]:
s46.get_closed_channels()

[<S46Channel: s2_A1 of S46: s2>,
 <S46Channel: s2_B1 of S46: s2>,
 <S46Channel: s2_R8 of S46: s2>]

In [4]:
len(s46.channels)

26

In [5]:
s46.A1.state("open")

In [6]:
s46.A1.state()

'open'

In [7]:
s46.A1.state("close")

In [8]:
s46.A1.state()

'close'

In [9]:
try: 
    s46.A2.state("close")
    raise("We should not be here")
except LockAcquisitionError as e: 
    print(e)

('Relay A is already in use by channel 1', 'setting s2_A2_state to close')


In [10]:
s46.A1.state("open")

In [11]:
s46.A2.state("close")

In [12]:
s46.A2.state("open")

In [13]:
assert s46.A1 is s46.channels[0]

In [14]:
assert s46.A2 is s46.channels[1]

In [15]:
assert s46.B1 is s46.channels[6]

In [16]:
s46.channels[6].short_name

'B1'

In [17]:
s46.A1.state("close")

In [18]:
s46.B1.state("close")

In [19]:
s46.R8.state("close")

In [20]:
s46.open_all_channels()

In [21]:
s46.A1.state("open")

In [70]:
channel_aliases = [
    f"{relay_name}{index}" for
    relay_name, index in product(
        ["A", "B", "C", "D"],
        range(1, 7)
    )
]
channel_aliases += [f"R{i}" for i in range(1, 9)]

print(dict(zip(channel_aliases, range(1, 33))))

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'A6': 6, 'B1': 7, 'B2': 8, 'B3': 9, 'B4': 10, 'B5': 11, 'B6': 12, 'C1': 13, 'C2': 14, 'C3': 15, 'C4': 16, 'C5': 17, 'C6': 18, 'D1': 19, 'D2': 20, 'D3': 21, 'D4': 22, 'D5': 23, 'D6': 24, 'R1': 25, 'R2': 26, 'R3': 27, 'R4': 28, 'R5': 29, 'R6': 30, 'R7': 31, 'R8': 32}


In [72]:
dict(
    zip(
        ["{}{}".format(*a) for a in product(["A", "B", "C", "D"], range(1, 7))] + [f"R{i}" for i in range(1, 9)], 
        range(1, 33)
    )
)

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'A6': 6,
 'B1': 7,
 'B2': 8,
 'B3': 9,
 'B4': 10,
 'B5': 11,
 'B6': 12,
 'C1': 13,
 'C2': 14,
 'C3': 15,
 'C4': 16,
 'C5': 17,
 'C6': 18,
 'D1': 19,
 'D2': 20,
 'D3': 21,
 'D4': 22,
 'D5': 23,
 'D6': 24,
 'R1': 25,
 'R2': 26,
 'R3': 27,
 'R4': 28,
 'R5': 29,
 'R6': 30,
 'R7': 31,
 'R8': 32}

In [45]:
relay_layout = [int(i) for i in s46.ask(":CONF:CPOL?").split(",")]
relay_names = (["A", "B", "C", "D"] + [f"R{i}" for i in range(1, 9)])
print(relay_layout)
print(relay_names)

[6, 6, 6, 6, 0, 0, 0, 0, 1, 0, 0, 1]
['A', 'B', 'C', 'D', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8']


In [47]:
class RelayLock: 
    def __init__(self, relay_name): 
        self._relay_name = relay_name
    
    def acquire(self, channel_number): 
        pass 
    
    def release(self, channel_number):
        pass 

In [51]:
class s46Channel: 
    def __init__(self, name, channel_number, lock): 
        self._name = name 
        self._lock = lock
        self._channel_number = channel_number

In [52]:
for relay_name, channel_count in zip(relay_names, relay_layout): 
    
    relay_lock = RelayLock(relay_name)
    
    for channel_index in range(1, channel_count + 1):
        
        if channel_count > 1:
            alias = f"{relay_name}{channel_index}"
        else:
            alias = relay_name
            
        channel_number = aliases[alias]
        channel = s46Channel(alias, channel_number, relay_lock)

In [56]:
class classproperty:
    def __init__(self, f):
        self.f = f
    def __get__(self, obj, owner):
        return self.f(owner)

In [61]:
class A: 
    @classmethod
    @property
    def aclassprop(cls): 
        return 1

In [62]:
A.aclassprop

<bound method ? of <class '__main__.A'>>